## 리그 오브 레전드 리그별 경기 결과 페이지 url 크롤링
  
https://qwer.gg/ko/leagues/{league}/{year}/{season}

+ 리그 : LCK(한국), LPL(중국), LCS(북미), LES(유럽)
+ 연도 : 2015 ~
+ 시즌 : 스프링, 서머

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
import pandas as pd
import time
import os
import warnings
warnings.filterwarnings("ignore")

In [3]:
options = webdriver.ChromeOptions()
#options.headless = True
options.add_argument('window_size=1920x1080')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36')

browser = webdriver.Chrome(options=options)

#qwer 사이트에서 LCK 2015년 서머시즌 페이지 열기
league = 'LCK'
year = '2016'
season = 'Summer'
url = f'https://qwer.gg/ko/leagues/{league}/{year}/{season}'

browser.get(url)

In [ ]:
#시즌 내 경기타입 선택
button = browser.find_element(By.ID, 'headlessui-menu-button-16')
button.click()

season_types = browser.find_element(By.ID, 'headlessui-menu-items-17').find_elements(By.TAG_NAME, 'div')
print(season_types[1].get_attribute('innerText'))

season_types[0].click()

Regular season


In [4]:
#경기결과 클릭
button = browser.find_element(By.ID, 'headlessui-tabs-tab-10')
button.click()   
time.sleep(3)

In [ ]:
#각 매치 결과 페이지 href 크롤링
buttons = browser.find_elements(By.TAG_NAME, 'button')

for i in reversed(buttons[16:]):   #매치 별 detail 버튼 누르기
    i.click()
    
a_tag = browser.find_elements(By.TAG_NAME, 'a')
match_link = [i.get_attribute('href') for i in a_tag][17:-5]

#txt 파일에 저장
filename = f'{league}_{year}_{season}_match_links.txt'
with open(filename, 'w') as file:
    file.write('\n'.join(match_link))

In [ ]:
#위 과정을 함수로 만들기
def get_match_url(league, year, season):
    for type_idx in range(4):
        options = webdriver.ChromeOptions()
        options.headless = True   #브라우저 창 띄우지 않고 백그라운드에서 진행
        options.add_argument('window_size=1920x1080')
        options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36')
        options.add_experimental_option("detach", True)
        browser = webdriver.Chrome(options=options)

        url = f'https://qwer.gg/ko/leagues/{league}/{year}/{season}'
        
        try:
            browser.get(url)
            time.sleep(3)
            
            button = browser.find_element(By.ID, 'headlessui-menu-button-16')  #상세시즌 사이드바
            button.click()
            
            season_types = browser.find_element(By.ID, 'headlessui-menu-items-17').find_elements(By.TAG_NAME, 'div')   #상세시즌 내역
                    
            season_type = season_types[type_idx]     
            season_name = season_type.get_attribute('innerText')  #txt 파일 저장 시 이름에 사용
                        
            season_type.click()
                
            button = browser.find_element(By.ID, 'headlessui-tabs-tab-10')
            button.click()   
            time.sleep(1)
                    
            buttons = browser.find_elements(By.TAG_NAME, 'button')

            for i in reversed(buttons[16:]):   #매치 별 detail 버튼 누르기
                i.click()
            time.sleep(1)
                    
            a_tag = browser.find_elements(By.TAG_NAME, 'a')
            match_link = [i.get_attribute('href') for i in a_tag][17:-5]

            #txt 파일에 저장
            filename = f'{league}_{year}_{season}_{season_name}_match_links.txt'
            with open(filename, 'w') as file:
                file.write('\n'.join(match_link))
            time.sleep(1)
            browser.quit()
        
        except:
            browser.quit()    

In [ ]:
#약 80분 소요
leagues = ['LCK', 'LPL', 'LCS', 'LEC']
years = [str(i) for i in range(2015, 2023)]
seasons = ['Spring', 'Summer']

for league in leagues:
    for year in years:
        for season in seasons:
            get_match_url(league, year, season)

In [ ]:
#폴더 내 txt 파일 모두 불러오는 함수
def get_txt_files(path):
    txt_file_list = [file for file in os.listdir(path) if file.endswith('txt')]
    return(txt_file_list)
    
txt_list = get_txt_files('./')    #현재 디렉토리 내 txt 파일들(116개)
len(txt_list)  

116

In [4]:
url = pd.read_csv('LCK_2019_Spring_Promotion_match_links.txt', header=None)

url_list = [i for i in url.iloc[:,0]]
url_list

['https://qwer.gg/ko/matches/16912/mvp-vs-btc-2018-09-22',
 'https://qwer.gg/ko/matches/16913/bbq-vs-mvp-2018-09-21',
 'https://qwer.gg/ko/matches/16914/dk-vs-btc-2018-09-20',
 'https://qwer.gg/ko/matches/16915/bbq-vs-dk-2018-09-18',
 'https://qwer.gg/ko/matches/16916/mvp-vs-btc-2018-09-18']

### 최종 결과 DataFrame 형식 생각해보기

In [60]:
# 아래 같은 형식으로 저장되도록 크롤링 함수 작성
import pandas as pd    
       
data1 = {'date' : ['2022-08-10']*5, 
        'vs' : ['GEN vs BRO']*5,
        'set' : ['1']*5,
        'player' : ['GEN Doran', 'Gen Peanut', 'Gen Chovy', 'Gen Ruler', 'Gen Lehends'],
        'champion' : ['Gwen', 'Leesin', 'Leblanc', 'Kalista', 'Amumu'], 
        'level' : ['16', '14', '18', '17', '14'], 
        'summoner_skill' : [['점멸', '순간이동'], 
                           ['강타', '점멸'], 
                           ['순간이동', '정화'], 
                           ['점멸', '정화'], 
                           ['점화', '점멸']],
        }

data2 = {'date' : ['2022-08-10']*5,
         'vs' : ['T1 vs LSB']*5,
        'set' : ['1']*5,
        'player' : ['T1 Zeus', 'T1 Oner', 'T1 Faker', 'T1 Gumayusi', 'T1 Keria'],
        'champion' : ['Sejuani', 'Pantheon', 'Azir', 'Jerri', 'Lulu'], 
        'level' : ['16', '14', '18', '17', '14'], 
        'summoner_skill' : [['점멸', '순간이동'], 
                           ['강타', '점멸'], 
                           ['순간이동', '정화'], 
                           ['점멸', '정화'], 
                           ['점화', '점멸']],
        }


df1 = pd.DataFrame(data1, 
                  columns=['date', 'vs', 'set', 'player', 'champion', 
                           'level', 'summoner_skill']).set_index('date')

df2 = pd.DataFrame(data2, 
                  columns=['date', 'vs', 'set', 'player', 'champion', 
                           'level', 'summoner_skill']).set_index('date')
df = pd.concat([df1, df2])
df

,vs,set,player,champion,level,summoner_skill
date,,,,,,
2022-08-10,GEN vs BRO,1,GEN Doran,Gwen,16,"[점멸, 순간이동]"
2022-08-10,GEN vs BRO,1,Gen Peanut,Leesin,14,"[강타, 점멸]"
2022-08-10,GEN vs BRO,1,Gen Chovy,Leblanc,18,"[순간이동, 정화]"
2022-08-10,GEN vs BRO,1,Gen Ruler,Kalista,17,"[점멸, 정화]"
2022-08-10,GEN vs BRO,1,Gen Lehends,Amumu,14,"[점화, 점멸]"
2022-08-10,T1 vs LSB,1,T1 Zeus,Sejuani,16,"[점멸, 순간이동]"
2022-08-10,T1 vs LSB,1,T1 Oner,Pantheon,14,"[강타, 점멸]"
2022-08-10,T1 vs LSB,1,T1 Faker,Azir,18,"[순간이동, 정화]"
2022-08-10,T1 vs LSB,1,T1 Gumayusi,Jerri,17,"[점멸, 정화]"


In [ ]:
#위 과정을 함수로 작성
def get_txt_url(path):
    txt_file_list = [file for file in os.listdir(path) if file.endswith('txt')]
    return(txt_file_list)
    
txt_list = get_txt_url('./')    #현재 디렉토리 내 txt 파일들(116개)

def get_league_data(url):
    options = webdriver.ChromeOptions()
    options.headless = True
    options.add_argument('window_size=1920x1080')
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36')
    browser = webdriver.Chrome(options=options)

    browser.get(url)
    
    final_data = []  #마지막 모든 데이터가 모일 list

    date = url[-10:]
    team1_name = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[1]/div[2]/div[1]/strong').get_attribute('innerText')
    team2_name = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[1]/div[2]/div[3]/strong').get_attribute('innerText')
    vs = team1_name + 'vs' + team2_name
    
    set_list = browser.find_elements(By.CLASS_NAME, 'shrink-0')  
    df_list = []
    for i in range(len(set_list)):
        #세트 선택
        set_num = set_list[i]
        set_num.click()
        
        #경기 영상 url
        videos = browser.find_element(By.XPATH, '//*[@id="__next"]/div[3]/div/div/div[1]/div[2]/div/div')
        video_urls = videos.find_elements(By.TAG_NAME, 'figure')
        video_url_list = []
        for i in video_urls:
            i.click()
            video_url_list.append(browser.find_element(By.TAG_NAME, 'iframe').get_attribute('src'))[i]
            
        #밴    
        banned = browser.find_elements(By.TAG_NAME, 'a')
        team1_banned = banned[25:30]
        team2_banned = banned[35:40]
        team1_banned = [banned_champ.find_elements(By.TAG_NAME, 'img') for banned_champ in team1_banned]
        team2_banned = [banned_champ.find_elements(By.TAG_NAME, 'img') for banned_champ in team2_banned]
        team1_banned = [banned_champ[0].get_attribute('src') for banned_champ in team1_banned]
        team2_banned = [banned_champ[0].get_attribute('src') for banned_champ in team2_banned]
        team1_banned = [str(banned_champ.split(sep='.')[-2]).split(sep='/')[-1] for banned_champ in team1_banned]*5
        team2_banned = [str(banned_champ.split(sep='.')[-2]).split(sep='/')[-1] for banned_champ in team2_banned]*5

        #오브젝트
        team1_object = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[2]/div[1]/div[2]')
        team1_object = team1_object.text.split(sep='\n')[1:]*5
        team2_object = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[2]/div[2]/div[2]')
        team2_object = team2_object.text.split(sep='\n')[1:]*5
        
        #획득 골드
        team1_gold = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[3]/div[2]/span[2]').get_attribute('innerText')*5
        team2_gold = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[3]/div[2]/span[3]').get_attribute('innerText')*5

        #KDA
        team1_kda = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[3]/div[1]/span[2]').get_attribute('innerText')*5
        team2_kda = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[1]/div[3]/div[3]/span[2]').get_attribute('innerText')*5

        #진영
        team1_side = browser.find_element(By.XPATH, '//*[@id="headlessui-tabs-panel-18"]/ul/li[1]/div[1]/div[2]/div/span').get_attribute('innerText')*5
        team2_side = browser.find_element(By.XPATH, '//*[@id="headlessui-tabs-panel-18"]/ul/li[2]/div[1]/div[2]/div/span').get_attribute('innerText')*5

        #승패 여부(victofy/defeated)
        team1_v_d = browser.find_element(By.XPATH, '//*[@id="headlessui-tabs-panel-18"]/ul/li[1]/div[1]/div[1]/strong').get_attribute('innerText')*5
        team2_v_d = browser.find_element(By.XPATH, '//*[@id="headlessui-tabs-panel-18"]/ul/li[2]/div[1]/div[1]/strong').get_attribute('innerText')*5

        #선수별 상세 데이터
        data = browser.find_element(By.CLASS_NAME, 'py-4').find_elements(By.TAG_NAME, 'tbody')
        player_data = [player_data.find_elements(By.TAG_NAME, 'tr') for player_data in data]   #(2 x 5)
        player_data_list = [x for x in player_data[0]] + [x for x in player_data[1]]           #(1 x 10)

        final_data = [] 

        for i in player_data_list:
            player_data = i.find_elements(By.TAG_NAME, 'td')[0]
            champion = str(player_data.find_element(By.TAG_NAME, 'img').get_attribute('src')).split('.')[-2].split('/')[-1]
            summoner_skill = sorted([i.find_elements(By.TAG_NAME, 'img')[1].get_attribute('title'), #.split(sep='.')[-2].split('/')[-1]
                    str(i.find_elements(By.TAG_NAME, 'img')[2].get_attribute('title'))]) #.split(sep='.')[-2].split('/')[-1]]
            rune = [str(i.find_elements(By.TAG_NAME, 'img')[3].get_attribute('alt')), #.split(sep='.')[-2].split('/')[-1]
                    str(i.find_elements(By.TAG_NAME, 'img')[4].get_attribute('alt'))] #.split(sep='.')[-2].split('/')[-1]
            
            player_data = i.get_attribute('innerText').split(sep='\n\t\n')
            level = player_data[0]
            player_name = player_data[1]
            
            kda = player_data[2].split('\n')[0]
            k, d, a = kda.split(' / ')
            
            damage = player_data[3]
            dpm = player_data[4]
            dtpm = player_data[5]
            sight = player_data[6].split(sep='\n')[0]
            ward_install = player_data[6].split(sep='\n')[1].split(sep=' / ')[0]
            ward_erase = player_data[6].split(sep='\n')[1].split(sep=' / ')[1]
            cs = player_data[7].split(sep='\n')[0]
            
            items = i.find_elements(By.TAG_NAME, 'td')[-1]
            items = items.find_elements(By.TAG_NAME, 'img')
            items = [item.get_attribute('alt') for item in items]
            
            final_data.append([champion, level, summoner_skill, rune, player_name, k, d, a, damage, dpm, dtpm, sight, cs, items])
        df_list.append(final_data)  

columns = ['date', 'vs', 'set', 'url', 'ban', 'object', 'team_gold', 'team_kda',
           'side', 'result', 'player', 'champion', 'level', 'summoner_skill', 'rune', 
           'level', 'k', 'd', 'a', 'damage', 'dpm', 'dtpm', 
           'sight_score', 'ward_install', 'ward_erase', 'cs', 'items']
        
